# 淘宝用户行为分析

## 项目简介
随着移动互联网技术的飞速发展和大众人均收入的增长，电商行业也随着技术的发展迅速崛起又快速分化。在当下大众消费需求日趋多样化的时代背景下，移动互联网行业特别是电商行业不再依靠用户红利实现业务增长，开始从粗放型的经营模式转向精细化管理，需要结合市场、渠道、用户行为等数据分析，对用户展开有针对性的运营活动，提供个性化、差异化的经营策略，从而实现运营目的。  
本项目使用Python分析淘宝平台2017年11月25日至2017年12月3日的用户行为数据，通过对网站流量、用户活跃率、用户行为转化漏斗、用户行为偏好等特征的分析，提供有针对性的运营策略。

## 数据理解

### 数据来源

本项目数据来源于阿里天池[User Behavior Data](https://tianchi.aliyun.com/dataset/dataDetail?dataId=649)，为了快速分析项目模型，项目仅截取完整数据集`UserBehavior`(3.41GB)的迷你数据集`UB_split_10.csv`(171.7MB)作为本项目的数据集。

### 数据属性
<table> 
    <tr>
        <td>
            <table border="1" cellspacing="5" cellpadding="5" align="left" style="text-align:left">
                <tr>
                    <th>变量名</th>
                    <th>中文释义</th>
                    <th>数据类型</th>
                </tr>    
                <tr>
                    <td >UserID</td>
                    <td>用户ID</td>
                    <td>整型</td>
                </tr>
                <tr>
                    <td>ItemID</td>
                    <td>商品ID</td>
                    <td>整型</td>
                </tr>
                <tr>
                    <td>CategoryID</td>
                    <td>商品所属种类ID</td>
                    <td>整型</td>
                </tr>
                <tr>
                    <td>CategoryID</td>
                    <td>商品所属种类ID</td>
                    <td>整型</td>
                </tr>
                <tr>
                    <td>BehaviorType</td>
                    <td>行为类型，包含pv,cart,buy和fav四种类型的行为</td>
                    <td>字符串</td>
                </tr>
                <tr>
                    <td>Timestamp</td>
                    <td>时间戳，用户发生某项行为当下时间点，精确到秒</td>
                    <td>整型</td>
                </tr>
            </table>
        </td>
     </tr>
    <tr> 
        <td>
            <table border="1" cellspacing="5" cellpadding="5" align="left" style="text-align:left">
                <tr>
                    <th>用户行为</th>
                    <th>解释说明</th>
                </tr>
                <tr>
                    <td>pv</td>
                    <td>商品浏览</td>
                </tr>
                <tr>
                    <td>cart</td>
                    <td>加入购物车</td>
                </tr>
                <tr>
                    <td>fav</td>
                    <td>收藏</td>
                </tr>
                <tr>
                    <td>buy</td>
                    <td>购买</td>
                </tr>
            </table>
        </td>
    </tr>
</table>

## 评估指标
<img src='.\评估指标.png'>

## 提出问题
通过上图的评估指标，项目分析主要解决以下问题：

- Q1. 网站流量情况：访问量PV、独立访客数UV、人均访问数PV/UV、跳出率（只存在浏览行为的访问量/PV）、DAU、活跃率等指标情况；
- Q2. 探索用户从浏览到购买整个过程的转化或者流失情况，从而确定关键夹点位置，为后续的改进提出意见；
- Q3. 探索用户的行为模式，从日期、时间、商品种类销量、具体商品销量等维度探索用户的行为偏好和模式；
- Q4. 挖掘用户价值，发现高价值用户

In [1]:
import pandas as pd
import os
from datetime import datetime,timedelta,timezone
import numpy as np
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True,threshold=1000000)

# 数据评估和清理
对数据集的基本信息，如数据类型、重复值、缺失值、异常值等评估和清理

## 错误数据类型处理
- 评估变量的数据类型是否存在错误

In [2]:
df = pd.read_csv('.\\拆分数据集\\UB_split_10.csv',names=['UserID','ItemID','CategoryID','BehaviorType','Timestamp'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 5 columns):
UserID          int64
ItemID          int64
CategoryID      int64
BehaviorType    object
Timestamp       int64
dtypes: int64(4), object(1)
memory usage: 171.7+ MB


In [4]:
df.head()

,UserID,ItemID,CategoryID,BehaviorType,Timestamp
0,314276,3657454,3738615,pv,1511572071
1,314276,98571,3738615,pv,1511572152
2,314276,76385,3108044,pv,1511572189
3,314276,1281342,3108044,pv,1511572236
4,314276,3893085,3607361,pv,1511572298


In [5]:
df.BehaviorType.unique()

array(['pv', 'cart', 'buy', 'fav'], dtype=object)

- 本项目共有500W条淘宝用户行为数据，每一行数据代表一个用户的行为记录
- 部分变量的数据类型有错误：
    - UserID、ItemID、CategoryID应改为string
    - BehaviorType改为Category更有效
    - Timestamp需改为datetime

In [6]:
# 按照上述评估结果，更正数据类型
df_copy = df.copy()
df_copy.UserID = df_copy.UserID.astype('str')
df_copy.ItemID = df_copy.ItemID.astype('str')
df_copy.CategoryID = df_copy.CategoryID.astype('str')
df_copy.BehaviorType = pd.Categorical(df_copy.BehaviorType)

In [7]:
# 将时间戳更改为日期时间型，注意这里需使用当地时间，不能使用utc时间
df_copy.Timestamp = pd.to_datetime(df_copy.Timestamp,unit='s')+timedelta(hours=8)

In [8]:
df_copy.head()

,UserID,ItemID,CategoryID,BehaviorType,Timestamp
0,314276,3657454,3738615,pv,2017-11-25 09:07:51
1,314276,98571,3738615,pv,2017-11-25 09:09:12
2,314276,76385,3108044,pv,2017-11-25 09:09:49
3,314276,1281342,3108044,pv,2017-11-25 09:10:36
4,314276,3893085,3607361,pv,2017-11-25 09:11:38


In [9]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 5 columns):
UserID          object
ItemID          object
CategoryID      object
BehaviorType    category
Timestamp       datetime64[ns]
dtypes: category(1), datetime64[ns](1), object(3)
memory usage: 100.1+ MB


## 异常值处理
- 探索行为发生的时间是否存在不合理时间点

In [10]:
end_time = df_copy.Timestamp.max()
start_time = df_copy.Timestamp.min()
print('起始时间：%s\n终止时间：%s'%(start_time,end_time))

起始时间：1918-08-01 12:37:51
终止时间：2020-11-02 07:24:47


In [11]:
# 查看非异常值的数据量
n_overtime = df_copy[(df_copy.Timestamp < '2017-11-25 00:00:00') | (df_copy.Timestamp >= '2017-12-04 00:00:00')].shape[0]

In [12]:
n_overtime/df_copy.shape[0]

0.0005212

项目分析是在2017年11月25日至2017年12月3日期间的用户行为记录，不在此事件范围内的数据量约占总数据量0.05%，对于分析结果影响甚微，将删除这部分数据记录

In [13]:
# 筛选符合时间范围内的数据记录
valid_df = df_copy[(df_copy.Timestamp >= '2017-11-25 00:00:00') & (df_copy.Timestamp < '2017-12-04 00:00:00')]

## 数据统一性处理
在处理数据类型时，观察到存在ItemID的数字位数不足7位，应该是源数据集提取时发生字符串转成数字后前面0消失的问题，所以这里需要对ItemID、UserID和CategoryID的数字位数做统一性处理

In [14]:
# 检查大部分ItemID、UserID和CategoryID的位数是几位
num_itemid = valid_df.ItemID.apply(lambda x: len(x))
num_userid = valid_df.UserID.apply(lambda x: len(x))
num_catid = valid_df.CategoryID.apply(lambda x:len(x))

In [15]:
num_itemid.value_counts()/5000000

7    0.806343
6    0.173391
5    0.017822
4    0.001745
3    0.000163
2    0.000014
Name: ItemID, dtype: float64

In [16]:
num_userid.value_counts()/5000000

6    0.900408
5    0.089358
4    0.008902
3    0.000766
2    0.000046
Name: UserID, dtype: float64

In [17]:
num_catid.value_counts()/5000000

7    0.823485
6    0.169200
5    0.006419
4    0.000375
Name: CategoryID, dtype: float64

In [18]:
# 在不足7位的ItemID前面填充0
valid_df.ItemID = valid_df.ItemID.str.zfill(7)
# 在不足6位的UserID前填充0
valid_df.UserID = valid_df.UserID.str.zfill(6)

In [19]:
# 对不足7位的CategoryID前面填充0
valid_df.CategoryID = valid_df.CategoryID.str.zfill(7)

## 数据完整性处理

In [20]:
# 根据Timestamp提取对应的日期、星期和小时变量(1-7分别代表了星期一至星期天)
valid_df['Date'] = valid_df.Timestamp.dt.date
valid_df['Hour'] = valid_df.Timestamp.dt.hour
valid_df['DayofWeek'] = valid_df.Timestamp.dt.dayofweek + 1

In [21]:
valid_df.head()

,UserID,ItemID,CategoryID,BehaviorType,Timestamp,Date,Hour,DayofWeek
0,314276,3657454,3738615,pv,2017-11-25 09:07:51,2017-11-25,9,6
1,314276,0098571,3738615,pv,2017-11-25 09:09:12,2017-11-25,9,6
2,314276,0076385,3108044,pv,2017-11-25 09:09:49,2017-11-25,9,6
3,314276,1281342,3108044,pv,2017-11-25 09:10:36,2017-11-25,9,6
4,314276,3893085,3607361,pv,2017-11-25 09:11:38,2017-11-25,9,6


In [ ]:
# 将清理好，用于本项目分析的数据集储存到本地
valid_df.to_csv('updated_valid_df.csv',index=False)